## Prediction rigidity calculation script for MACE

IMPORTANT TO INSTALL AND USE THE MACE VERSION AVAILABLE HERE:
<https://github.com/SanggyuChong/mace/tree/LLPR_farad>

In [ ]:
import numpy as np
import torch
import torch.nn.functional

from mace import data, modules, tools
from mace.tools import torch_geometric
from mace.tools.scripts_utils import get_dataset_from_xyz

import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64)
device = tools.init_device('cuda')

### load model

In [ ]:
mace = torch.load('MACE_model.model').to('cuda')
mace_llpr = modules.LLPRModel(mace)
mace_llpr.to(device);

In [ ]:
stats = {"atomic_numbers": [14],
         "r_max": 5.5}
config_type_weights = {"Default": 1.0}
z_table = tools.get_atomic_number_table_from_zs(stats['atomic_numbers'])

### load datasets

In [ ]:
collections, atomic_energies_dict = get_dataset_from_xyz(
    train_path="train.xyz",
    valid_path="valid.xyz",
    test_path="test.xyz",
    valid_fraction=0,
    config_type_weights=config_type_weights,
    energy_key="free_energy",
)

In [ ]:
train_loader = torch_geometric.dataloader.DataLoader(
    dataset=[
        data.AtomicData.from_config(config, z_table=z_table, cutoff=stats['r_max'])
        for config in collections.train
    ],
    batch_size=10,
    shuffle=False,
    drop_last=False,
)

valid_loader = torch_geometric.dataloader.DataLoader(
    dataset=[
        data.AtomicData.from_config(config, z_table=z_table, cutoff=stats['r_max'])
        for config in collections.valid
    ],
    batch_size=10,
    shuffle=False,
    drop_last=False,
)

test_loader = torch_geometric.dataloader.DataLoader(
    dataset=[
        data.AtomicData.from_config(config, z_table=z_table, cutoff=stats['r_max'])
        for config in collections.tests[0][1]
    ],
    batch_size=10,
    shuffle=False,
    drop_last=False,
)

### compute covariance and inv covariance on original training set

In [ ]:
mace_llpr.compute_covariance(train_loader)
mace_llpr.compute_inv_covariance(C = 1, sigma = 5e-6)

In [ ]:

test_PRs = []
test_LPRs = []

for batch in iter(test_loader):

    batch.to(device)
    outputs = mace_llpr(batch, save_atomic_llfeats=True)
    
    test_PRs.append(1 / outputs['energy_uncertainty'].cpu().detach().numpy())

    ll_feats = outputs["atomic_llfeats"]

    cur_LPRs = 1 / torch.einsum(
        "ij, jk, ik -> i",
        ll_feats,
        mace_llpr.inv_covariance,
        ll_feats,
    )
    
    test_LPRs.append(cur_LPRs.cpu().detach().numpy())
    

In [ ]:
test_pr = np.hstack(test_PRs)
test_lpr = np.hstack(test_LPRs)